<a href="https://colab.research.google.com/github/lguloy/House-Pricing-Predictions/blob/House_Price_ML_CCG/Dummies_House_Price_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [473 kB]
Get:13 htt

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Import RandomForestRegressor and Pickle

from sklearn.ensemble import RandomForestRegressor
import pickle

In [13]:
#Load model
#NOTE: Must manually load file before un-pickling

model = pickle.load(open("VLC_Ames_Housing_Price_Model", 'rb'))

In [ ]:
# Load in predict.csv from S3 into a DataFrame
url = "https://finalprojectbootcamp.s3.us-east-2.amazonaws.com/test_converted_df.csv"
spark.sparkContext.addFile(url)

dfp = spark.read.option('header', 'true').csv(SparkFiles.get("test_converted_df.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
dfp.show(10)

#Convert to Pandas to allow train_test_split usage
dfp = df.select("*").toPandas()
dfp = df.dropna(how="any")

In [ ]:
X_predict = dfp[["1stFlrSF", 
        "GrLivArea", 
        "LotArea", 
        "GarageArea", 
        "BsmtUnfSF", 
        "TotalBsmtSF", 
        "LotFrontage", 
        "GarageYrBlt", 
        "MoSold", 
        "YearBuilt"]]

y_predict = model.predict(X_predict)

for i in X_predict:
	print("Predicted=%s" % (y_predict[i]))